<a href="https://colab.research.google.com/github/oraziotorre/MomentumShiftAI/blob/main/ModelDevelopment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [94]:
# Pandas per la gestione dei dati
import pandas as pd
import requests

from collections import Counter

# Per visualizzare i DataFrame su Google Colab in formato tabellare
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [95]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2020s.csv"
response = requests.get(url)

with open("charting-m-points-2020s.csv", "w") as f:
    f.write(response.text)

dataset1 = pd.read_csv("charting-m-points-2020s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-2010s.csv"
response = requests.get(url)

with open("charting-m-points-2010s.csv", "w") as f:
    f.write(response.text)

dataset2 = pd.read_csv("charting-m-points-2010s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_MatchChartingProject/master/charting-m-points-to-2009.csv"
response = requests.get(url)

with open("charting-m-points-to-2009.csv", "w") as f:
    f.write(response.text)

dataset3 = pd.read_csv("charting-m-points-to-2009.csv")


dataset = pd.concat([dataset1, dataset2, dataset3], ignore_index=True)

dataset

<ipython-input-95-98acf44c7188>:25: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  dataset3 = pd.read_csv("charting-m-points-2020s.csv")


,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731734,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,5,2.0,15-0,31,True,1,6w,5b39b2f1f1f2b2b1f1f2f2b2d@,NaN,1
731735,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,5,2.0,30-0,31,True,1,4n,4f28f3b2f1f2f+1r2v3*,NaN,1
731736,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,5,2.0,40-0,31,True,1,6f28b2b3b;2f1f1f3b;3s1f3w@,NaN,NaN,2
731737,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,5,2.0,40-15,31,True,1,5b37b2b1f2f2b2d@,NaN,NaN,2


Numero di partite del dataset

In [96]:
simboli_finali = dataset['match_id'].nunique()
simboli_finali

4497

Analisi delle tipologie degli esiti finali dei punti, analisi dei casi rari

In [97]:
simboli_finali = Counter()


for index,row in dataset.iterrows():
  if(pd.isna(row['2nd'])):
    ultimo_carattere = row['1st'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['1st'][-2]
    simboli_finali[ultimo_carattere] += 1

  else:
    ultimo_carattere = row['2nd'][-1]
    if(ultimo_carattere == ";"):
      ultimo_carattere = row['2nd'][-2]
    simboli_finali[ultimo_carattere] += 1




for simbolo, conteggio in simboli_finali.most_common():
    print(f"'{simbolo}': {conteggio}")

'#': 242297
'@': 233365
'*': 227917
'n': 10985
'd': 10812
'w': 2965
'S': 1479
'R': 690
'x': 649
'C': 288
'g': 52
'!': 43
'Q': 42
' ': 33
'P': 30
'3': 20
'e': 18
'1': 16
'9': 6
'2': 6
'8': 6
'7': 4
'+': 3
'c': 2
'^': 2
'b': 2
'p': 1
'.': 1
'-': 1
'4': 1
'&': 1
')': 1
'v': 1


Correzione dei casi rari e enfasi a informazioni importanti in un formato più gestibile.

La descrizione in formato semplice, una riga per ogni stringa:


- `1` = `noinfo_1`: il giocatore 1 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `2` = `noinfo_2`: il giocatore 2 vince il punto, ma non è chiaro se è ace, errore, o altro.

- `3` = `serve_ace_1`: punto diretto con il servizio (ace) fatto dal giocatore 1.

- `4` = `serve_ace_2`: punto diretto con il servizio (ace) fatto dal giocatore 2.

- `5` = `serve_miss2_1`: il giocatore 1 vince il punto su un errore di servizio del giocatore 2.

- `6` = `serve_miss1_2`: il giocatore 2 vince il punto su un errore di servizio del giocatore 1.

- `7` = `rally_winner_1`: il giocatore 1 vince il punto con un colpo vincente durante lo scambio.

- `8` = `rally_winner_2`: il giocatore 2 vince il punto con un colpo vincente durante lo scambio.

- `9` = `rally_forced2_1`: il giocatore 1 forza un errore del giocatore 2 durante lo scambio.

- `A` = `rally_forced1_2`: il giocatore 2 forza un errore del giocatore 1 durante lo scambio.

- `B` = `rally_unforced1_2`: il giocatore 2 vince per un errore non forzato del giocatore 1.

- `C` = `rally_unforced2_1`: il giocatore 1 vince per un errore non forzato del giocatore 2.

Attualmente non ho aggiunto considerazioni sul primo e secondo servizio, e su posizionamenti della palla e tipologia di colpi giocati.

Ho considerato le challenges Hawk-Eye perse come errori forzati.
Errori forzati e winners sono stati considerati come la stessa cosa nei servizi, si potrebbe fare la stessa considerazione anche nei rally

In [98]:
for index, row in dataset.iterrows():
    winner = row["PtWinner"]

    #On first serve points
    if pd.isna(row['2nd']):
        numeri = [c for c in row['1st'] if c.isdigit()]
        ultimo = row['1st'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['1st'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'

    #On second serve points
    else:
        numeri = [c for c in row['2nd'] if c.isdigit()]
        ultimo = row['2nd'][-1]
        if(ultimo_carattere == ";"):
          ultimo_carattere = row['2nd'][-2]

        #On serve points
        if len(numeri) == 1:
            if (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 1:
                dataset.at[index, 'PointType'] = '3'
            elif (ultimo == '#' or ultimo == '*' or ultimo == 'C') and winner == 2:
                dataset.at[index, 'PointType'] = '4'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 1:
                dataset.at[index, 'PointType'] = '5'
            elif (ultimo == '@' or ultimo == 'd' or ultimo == 'w' or ultimo == 'n' or ultimo == 'x'or ultimo == 'g') and winner == 2:
                dataset.at[index, 'PointType'] = '6'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                else:
                  dataset.at[index, 'PointType'] = '2'

        #On rally points
        else:
            if ultimo == '*' and winner == 1:
                dataset.at[index, 'PointType'] = '7'
            elif ultimo == '*' and winner == 2:
                dataset.at[index, 'PointType'] = '8'
            elif (ultimo == '#' or ultimo =="C") and winner == 1:
                dataset.at[index, 'PointType'] = '9'
            elif (ultimo == '#' or ultimo =="C") and winner == 2:
                dataset.at[index, 'PointType'] = 'A'
            elif ultimo == '@' and winner == 1:
                dataset.at[index, 'PointType'] = 'B'
            elif ultimo == '@' and winner == 2:
                dataset.at[index, 'PointType'] = 'C'
            else:
                if winner == 1:
                    dataset.at[index, 'PointType'] = '1'
                elif winner == 2:
                  dataset.at[index, 'PointType'] = '2'


dataset

,match_id,Pt,Set1,Set2,Gm1,Gm2,Pts,Gm#,TbSet,Svr,1st,2nd,Notes,PtWinner,PointType
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,1,0,0,0,0.0,0-0,1,True,1,4n,4b1b1b3n@,NaN,1,B
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,2,0,0,0,0.0,15-0,1,True,1,5n,5f28f2b1b1b2f3f3d@,NaN,1,B
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,3,0,0,0,0.0,30-0,1,True,1,4C,NaN,NaN,1,3
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,4,0,0,0,0.0,40-0,1,True,1,4r2d#,NaN,NaN,1,9
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,5,0,0,1,0.0,0-0,2,True,2,5d,c4b1b1d!@,NaN,1,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
731734,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,203,1,1,5,2.0,15-0,31,True,1,6w,5b39b2f1f1f2b2b1f1f2f2b2d@,NaN,1,B
731735,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,204,1,1,5,2.0,30-0,31,True,1,4n,4f28f3b2f1f2f+1r2v3*,NaN,1,7
731736,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,205,1,1,5,2.0,40-0,31,True,1,6f28b2b3b;2f1f1f3b;3s1f3w@,NaN,NaN,2,C
731737,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,206,1,1,5,2.0,40-15,31,True,1,5b37b2b1f2f2b2d@,NaN,NaN,2,C


In [99]:
simboli_finali = dataset['PointType'].value_counts()

simboli_finali

,count
PointType,
C,116151
B,114569
9,105469
A,102281
7,83304
8,82725
3,50088
4,46635
6,13854


Creazione delle progressioni punteggio

In [100]:
new_data = []
current_id = None
current_set = None
current_game = None
last_point = None
new_row = None

for index, row in dataset.iterrows():
    match_id = row['match_id']
    set_score = row['Set1'] + row['Set2']
    game_score = row['Gm1'] + row['Gm2']
    point_winner = row['PtWinner']
    point_type = row['PointType']

    # If New match
    if match_id != current_id:
        # Save previous Match labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point
              if r['MatchWinner'] == 0:
                  r['MatchWinner'] = last_point


        current_id = match_id
        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            "Svr": row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New set of the same Match
    elif set_score != current_set:
        # Save previous Set labels
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)
            for r in new_data:
              if r['SetWinner'] == 0:
                  r['SetWinner'] = last_point



        current_set = set_score
        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If New game of the same Set
    elif game_score != current_game:
        # Save previous Game label
        if new_row and last_point:
            new_row['GmWinner'] = last_point
            new_data.append(new_row)


        current_game = game_score
        new_row = {
            'match_id': match_id,
            'GmProgression': [point_winner],
            'PoweredGmProgression': [point_type],
            'Svr': row['Svr'],
            'Set1': row['Set1'],
            'Set2': row['Set2'],
            'Gm1': row['Gm1'],
            'Gm2': int(row['Gm2']) if not pd.isna(row['Gm2']) else 0,
            'GmWinner': 0,
            'SetWinner': 0,
            'MatchWinner': 0
        }

    # If same Game, Set and Match of the previous consideration
    else:
        # Save Game progression sequences
        if new_row:
            new_row['GmProgression'].append(point_winner)
            new_row['PoweredGmProgression'].append(point_type)

    last_point = point_winner

# Last dataset's game memorization
if new_row and last_point:
    new_row['GmWinner'] = last_point
    new_data.append(new_row)
    for r in new_data:
      if r['SetWinner'] == 0:
          r['SetWinner'] = last_point
      if r['MatchWinner'] == 0:
          r['MatchWinner'] = last_point




new_dataset = pd.DataFrame(new_data)
new_dataset

,match_id,GmProgression,PoweredGmProgression,Svr,Set1,Set2,Gm1,Gm2,GmWinner,SetWinner,MatchWinner
0,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]","[B, B, 3, 9]",1,0,0,0,0,1,1,1
1,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 2, 2, 1, 2, 2, 2]","[B, B, 4, A, 9, 8, A, A]",2,0,0,1,0,2,1,1
2,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[2, 1, 1, 1, 1]","[8, 7, B, 7, 3]",1,0,0,1,1,1,1,1
3,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 2, 2, 1, 1, 2, 2, 2]","[B, A, 8, 7, B, 4, 8, 8]",2,0,0,2,1,2,1,1
4,20191124-M-Davis_Cup_Finals-F-Rafael_Nadal-Den...,"[1, 1, 1, 1]","[9, 3, 3, 3]",1,0,0,2,2,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
118018,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,"[1, 1, 1, 2, 2, 1]","[3, 7, 9, A, C, 9]",1,1,1,3,0,1,1,1
118019,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,"[1, 1, 2, 1, 2, 2, 1, 2, 2, 2]","[B, 7, 8, 9, A, 8, B, 8, C, 8]",2,1,1,4,0,2,1,1
118020,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,"[1, 1, 1, 2, 1]","[7, 9, B, C, B]",1,1,1,4,1,1,1,1
118021,20200103-M-ATP_Cup-RR-Alex_De_Minaur-Alexander...,"[2, 2, 1, 1, 2, 1, 2, 1, 2, 2]","[C, A, 7, 9, A, B, 8, 9, 4, A]",2,1,1,5,1,2,1,1


Analisi Bilanciamento Etichette

In [101]:
simboli_finali = new_dataset['GmWinner'].value_counts()

simboli_finali

,count
GmWinner,
1,59948
2,58075


In [102]:
simboli_finali = new_dataset['SetWinner'].value_counts()

simboli_finali

,count
SetWinner,
1,60129
2,57894


In [103]:
simboli_finali = new_dataset['MatchWinner'].value_counts()

simboli_finali

,count
MatchWinner,
1,60319
2,57704


Prendo la lista dei giocatori di Tennis

In [144]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_players.csv"
response = requests.get(url)

with open("atp_players.csv", "w") as f:
    f.write(response.text)

playerset = pd.read_csv("atp_players.csv")
playerset['full_name'] = (
    playerset['name_first']
    .str.replace(' ', '_', regex=False)
    .str.replace('-', '_', regex=False)
    + '_' +
    playerset['name_last']
    .str.replace(' ', '_', regex=False)
    .str.replace('-', '_', regex=False)
)
playerset

<ipython-input-144-4b2e25370eee>:7: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  playerset = pd.read_csv("atp_players.csv")


,player_id,name_first,name_last,hand,dob,ioc,height,wikidata_id,full_name
0,100001,Gardnar,Mulloy,R,19131122.0,USA,185.0,Q54544,Gardnar_Mulloy
1,100002,Pancho,Segura,R,19210620.0,ECU,168.0,Q54581,Pancho_Segura
2,100003,Frank,Sedgman,R,19271002.0,AUS,180.0,Q962049,Frank_Sedgman
3,100004,Giuseppe,Merlo,R,19271011.0,ITA,NaN,Q1258752,Giuseppe_Merlo
4,100005,Richard,Gonzalez,R,19280509.0,USA,188.0,Q53554,Richard_Gonzalez
...,...,...,...,...,...,...,...,...,...
65984,213700,Matvei,Kobiakov,U,NaN,RUS,NaN,NaN,Matvei_Kobiakov
65985,213701,Tobia Costanzo,Baragiola Mordini,U,NaN,ITA,NaN,NaN,Tobia_Costanzo_Baragiola_Mordini
65986,213702,Dominik,Wijntjes,U,NaN,NZL,NaN,NaN,Dominik_Wijntjes
65987,213703,Sam,Wensley,U,NaN,AUS,NaN,NaN,Sam_Wensley


Prendo i rankings degli ultimi 20 anni

In [164]:
url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_current.csv"
response = requests.get(url)

with open("atp_rankings_70s.csv", "w") as f:
    f.write(response.text)

ranking1 = pd.read_csv("atp_rankings_70s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_20s.csv"
response = requests.get(url)

with open("atp_rankings_80s.csv", "w") as f:
    f.write(response.text)

ranking2 = pd.read_csv("atp_rankings_80s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_10s.csv"
response = requests.get(url)

with open("atp_rankings_90s.csv", "w") as f:
    f.write(response.text)

ranking3 = pd.read_csv("atp_rankings_90s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_00s.csv"
response = requests.get(url)

with open("atp_rankings_00s.csv", "w") as f:
    f.write(response.text)

ranking4 = pd.read_csv("atp_rankings_00s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_90s.csv"
response = requests.get(url)

with open("atp_rankings_10s.csv", "w") as f:
    f.write(response.text)

ranking5 = pd.read_csv("atp_rankings_10s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_80s.csv"
response = requests.get(url)

with open("atp_rankings_20s.csv", "w") as f:
    f.write(response.text)

ranking6 = pd.read_csv("atp_rankings_20s.csv")


url = "https://raw.githubusercontent.com/JeffSackmann/tennis_atp/master/atp_rankings_70s.csv"
response = requests.get(url)

with open("atp_rankings_current.csv", "w") as f:
    f.write(response.text)

ranking7 = pd.read_csv("atp_rankings_current.csv")

rankingset = pd.concat([ranking1, ranking2, ranking3, ranking4, ranking5, ranking6, ranking7], ignore_index=True)

rankingset

,ranking_date,rank,player,points
0,20240101,1,104925,11245.0
1,20240101,2,207989,8855.0
2,20240101,3,106421,7600.0
3,20240101,4,206173,6490.0
4,20240101,5,126094,4805.0
...,...,...,...,...
3292944,19771212,95,100586,NaN
3292945,19771212,97,100363,NaN
3292946,19771212,98,100176,NaN
3292947,19771212,99,100233,NaN


Dataset con i ranking dei giocatori

In [ ]:
player_id_map = playerset.set_index('full_name')['player_id'].astype(str).str.lower().to_dict()
last_match = None
ranking_1 = None
ranking_2 = None

def extract_rankings(row):
    global last_match, ranking_1, ranking_2
    match_id = row['match_id']
    if match_id == last_match:
      return pd.Series([ranking_1, ranking_2])
    else:
      last_match = match_id

      match_info = match_id.split('-')
      ranking_date = match_info[0]
      name1 = match_info[-2].lower()
      name2 = match_info[-1].lower()

      # Trova i player_id in modo sicuro
      player_id1 = player_id_map.get(name1)
      player_id2 = player_id_map.get(name2)


      filtered_rankings = rankingset[rankingset['ranking_date'].astype(str) <= ranking_date]
      for index, row in filtered_rankings.iterrows():
          if row['player'] == player_id1:
              ranking_1 = row['rank']
              break

      for index, row in filtered_rankings.iterrows():
          if row['player'] == player_id2:
              ranking_2 = row['rank']
              break

      return pd.Series([ranking_1, ranking_2])


new_dataset[['Ranking1', 'Ranking2']] = new_dataset.apply(extract_rankings, axis=1)
new_dataset